In this project, as input data we have some dummy data (in sqlite format) that refer to a three month log of the clicks in a e-shop. The data include customer ids, item and category ids, and the duration (in seconds) that each customer spent when clicking on an item, each time. Regular customers have ids starting with "CC" prefix. 

Using this data, we will create a list where we will recommend ten products to each customer that would most probable interest them. 

For the recommendation list we will apply different models to the original duration data, the dummy data ('1' if a customer clicked on an item) and the scaled duration data. The models we will use are: content-based popularity model and collaborative filtering (cosine similarity and pearson coefficient) and we will choose the one which gives the best results.

In addition, we will recommend the ten most suitable items to a specific given customer.

Last, we will produce a list of the customers that would be interested in items of a specific category (e.g. so that they can be informed if the e-shop wishes to promote a new or existing item of this category).

## 1. Import modules

In [1]:
import pandas as pd
import sqlite3

In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

## 2. Load data

In [1]:
# Load the data from all .sqlite files in one dataframe for further analysis and manipulation

In [3]:
months = {3:'March', 4:'April', 5:'May'}

month_df = pd.DataFrame()

for k,v in months.items():
    for i in range(1,31+k%2,1):
        cnx = sqlite3.connect(r'../Data'+'//'+ v+'//2019_' + v + '_' + str(i)+'.sqlite')
        #print(r'../Data'+'//'+ v+'//2019_' + v + '_' + str(i)+'.sqlite')
        #print("Connection Successful",cnx)
        df1 = pd.read_sql('select * from Data',cnx)
        cnx.close()
        # add a column to indicate the day
        df1['Day'] = str(i)
        # add a column to indicate the month
        df1['Month'] = str(k)
        month_df = month_df.append(df1)

print(month_df.shape)

(576278, 3)


In [4]:
df = month_df

# split the information from the Sess_Info column
df[['Customer', 'Item', 'Category', 'SecNum', 'Sec']] = month_df['Sess_Info'].str.split('_',expand=True)

# columns Sec and Sess_Info are of no use
df = df.drop('Sec',axis=1)
df = df.drop('Sess_Info',axis=1)

# remove the prefix 'item' from column Item and the prefix 'cat' from column Category
df = df.apply(lambda S:S.str.strip('item'))
df = df.apply(lambda S:S.str.replace('cat',''))

# add a column with the Number of seconds as floats for later use
df['SecNumF'] = df['SecNum'].astype(float)

In [5]:
print(df.shape)
#df.info()
#df.describe()
df.columns
#df.head

(576278, 7)


Index([u'Day', u'Month', u'Customer', u'Item', u'Category', u'SecNum',
       u'SecNumF'],
      dtype='object')

## 3. Data preparation

#### 3.1. Create data with user, item, and target field (duration of clicks in seconds

In [6]:
# I will use only the customers, the items, the categories and the number of seconds of each click.

df2 = df[['Customer', 'Item', 'Category', 'SecNumF']]

In [7]:
# Since each specific product is determined by a combination of an ItemId and a CategoryId, 
# I will create a unique productId as a concatenation of the category and the item id.

# Product = CategoryId_ItemId

df3 = df2.assign(Product = df2.Category.astype(str) + '_' + df2.Item.astype(str)).drop(['Item', 'Category'], axis=1)
df3.head()

,Customer,SecNumF,Product
0,CC201903212,11.0,3_16
1,CC201903212,12.0,6_16
2,CC201903212,8.0,4_1
3,CC201903212,27.0,4_10
4,CC201903212,27.0,5_2


In [8]:
# Calculate the total sum of the time each customer spent in each product.

data = df3.groupby(['Customer','Product'])['SecNumF'].sum().reset_index()

data.head()
#data.columns

,Customer,Product,SecNumF
0,CC20190310,0_0,146.0
1,CC20190310,0_1,8.0
2,CC20190310,0_10,89.0
3,CC20190310,0_11,33.0
4,CC20190310,0_12,38.0


#### 3.2. Create dummy

In [9]:
# If a customer has clicked on a product, then SecNumF_Dummy is marked as 1

def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['SecNumF_Dummy'] = 1
    return data_dummy

data_dummy = create_data_dummy(data)

#### 3.3. Normalize item values across users

In [10]:
# create a user-item matrix

df_matrix = pd.pivot_table(data, values='SecNumF', index='Customer', columns='Product')

In [11]:
# normalize time of clicks of each product across customers
# from 0–1 (with 1 being the highest number of seconds for an item and 0 being 0 time for that item)

df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [12]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['Scaled_SecNumF'] 
data_norm = pd.melt(d, id_vars=['Customer'], value_name='Scaled_SecNumF').dropna()
print(data_norm.shape)
data_norm.head()

(178122, 3)


,Customer,Product,Scaled_SecNumF
0,CC20190310,0_0,0.437309
1,CC201903100,0_0,0.250765
2,CC201903101,0_0,0.055046
3,CC201903102,0_0,0.223242
4,CC201903103,0_0,0.538226


## 4. Split train and test set

In [15]:
# We use 80:20 ratio for our train-test set size.
# Our training portion will be used to develop a predictive model, while the other to evaluate the model’s performance.

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [16]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Define Models using Turicreate library

In [18]:
# define our variables to use in the models
# constant variables to define field names include:

user_id = 'Customer'
item_id = 'Product'
users_to_recommend = list(data[user_id])
n_rec = 10       # number of items to recommend
n_display = 30   # to display the first few rows in an output dataset

In [19]:
# define a function for all models

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

## 6. Popularity Model

In [20]:
# Using number of seconds SecNumF

name = 'popularity'
target = 'SecNumF'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.296295s

142497 observations to process; with 235 unique items.

recommendations finished on 1000/178122 queries. users per second: 181554

...

recommendations finished on 175000/178122 queries. users per second: 89268.2

recommendations finished on 176000/178122 queries. users per second: 89148.2

recommendations finished on 177000/178122 queries. users per second: 89029

+------------+---------+---------------+------+
|  Customer  | Product |     score     | rank |
+------------+---------+---------------+------+
| CC20190310 |   2_15  | 56.0674846626 |  1   |
| CC20190310 |   3_21  | 55.9212962963 |  2   |
| CC20190310 |   2_29  | 55.8069306931 |  3   |
| CC20190310 |   2_10  | 55.5778491172 |  4   |
| CC20190310 |   0_15  | 55.5538461538 |  5   |
| CC20190310 |   2_28  | 55.3065326633 |  6   |
| CC20190310 |   1_21  | 55.2561349693 |  7   |
| CC20190310 |   7_9   | 55.0095238095 |  8   |
| CC20190310 |   6_19  | 54.9495114007 |  9   |
| CC20190310 |   5_21  | 54.8238021638 |  10  |
| CC20190310 |   2_15  | 56.0674846626 |  1   |
| CC20190310 |   3_21  | 55.9212962963 |  2   |
| CC20190310 |   2_29  | 55.8069306931 |  3   |
| CC20190310 |   2_10  | 55.5778491172 |  4   |
| CC20190310 |   0_15  | 55.5538461538 |  5   |
| CC20190310 |   2_28  | 55.3065326633 |  6   |
| CC20190310 |   1_21  | 55.2561349693 |  7   |
| CC20190310 |   7_9   | 55.0095238095 |

recommendations finished on 178000/178122 queries. users per second: 89004.9

In [21]:
# Using SecNumF dummy

name = 'popularity'
target = 'SecNumF_Dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns SecNumF;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.388144s

...

recommendations finished on 174000/178122 queries. users per second: 87367.1

recommendations finished on 175000/178122 queries. users per second: 87274.9

recommendations finished on 176000/178122 queries. users per second: 87312.5

recommendations finished on 177000/178122 queries. users per second: 87404.1

recommendations finished on 178000/178122 queries. users per second: 87492.3

In [22]:
# Using scaled number of seconds

name = 'popularity'
target = 'Scaled_SecNumF'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.282958s

142497 observations to process; with 235 unique items.

recommendations finished on 1000/178122 queries. users per second: 151194

...

recommendations finished on 174000/178122 queries. users per second: 92838.1

recommendations finished on 175000/178122 queries. users per second: 92303.8

recommendations finished on 176000/178122 queries. users per second: 92319.2

recommendations finished on 177000/178122 queries. users per second: 92192.7

recommendations finished on 178000/178122 queries. users per second: 90878.7

## 7. Collaborative Filtering Model

#### 7.1. Cosine similarity

In [23]:
# Using number of seconds

name = 'cosine'
target = 'SecNumF'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.27094s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 175000/178122 queries. users per second: 19890.3

recommendations finished on 176000/178122 queries. users per second: 19831.8

recommendations finished on 177000/178122 queries. users per second: 19675.5

recommendations finished on 178000/178122 queries. users per second: 19522.7

+------------+---------+---------------+------+
|  Customer  | Product |     score     | rank |
+------------+---------+---------------+------+
| CC20190310 |   5_4   | 39.0376383932 |  1   |
| CC20190310 |   4_14  | 29.1086889101 |  2   |
| CC20190310 |   5_3   | 22.9814542333 |  3   |
| CC20190310 |   3_21  | 18.6703356653 |  4   |
| CC20190310 |   5_28  | 16.3121008976 |  5   |
| CC20190310 |   1_27  | 15.3230272879 |  6   |
| CC20190310 |   6_23  | 14.3274597731 |  7   |
| CC20190310 |   5_15  | 12.8312043285 |  8   |
| CC20190310 |   2_16  |  12.103432391 |  9   |
| CC20190310 |   7_27  | 12.0736098492 |  10  |
| CC20190310 |   5_4   | 39.0376383932 |  1   |
| CC20190310 |   4_14  | 29.1086889101 |  2   |
| CC20190310 |   5_3   | 22.9814542333 |  3   |
| CC20190310 |   3_21  | 18.6703356653 |  4   |
| CC20190310 |   5_28  | 16.3121008976 |  5   |
| CC20190310 |   1_27  | 15.3230272879 |  6   |
| CC20190310 |   6_23  | 14.3274597731 |  7   |
| CC20190310 |   5_15  | 12.8312043285 |

In [24]:
# Using SecNumF dummy

name = 'cosine'
target = 'SecNumF_Dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns SecNumF;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.285269s

...

recommendations finished on 175000/178122 queries. users per second: 19408.1

recommendations finished on 176000/178122 queries. users per second: 19331.9

recommendations finished on 177000/178122 queries. users per second: 19273.7

recommendations finished on 178000/178122 queries. users per second: 19238.2

+------------+---------+----------------+------+
|  Customer  | Product |     score      | rank |
+------------+---------+----------------+------+
| CC20190310 |   7_3   | 0.429502633929 |  1   |
| CC20190310 |   4_8   | 0.334763519248 |  2   |
| CC20190310 |   2_4   | 0.318219643239 |  3   |
| CC20190310 |   4_19  | 0.29304766411  |  4   |
| CC20190310 |   7_9   | 0.289801733536 |  5   |
| CC20190310 |   5_4   | 0.287754414374 |  6   |
| CC20190310 |   0_14  | 0.262947566677 |  7   |
| CC20190310 |   2_19  | 0.253219620875 |  8   |
| CC20190310 |   4_6   | 0.247925456853 |  9   |
| CC20190310 |   1_5   | 0.246392171634 |  10  |
| CC20190310 |   7_3   | 0.429502633929 |  1   |
| CC20190310 |   4_8   | 0.334763519248 |  2   |
| CC20190310 |   2_4   | 0.318219643239 |  3   |
| CC20190310 |   4_19  | 0.29304766411  |  4   |
| CC20190310 |   7_9   | 0.289801733536 |  5   |
| CC20190310 |   5_4   | 0.287754414374 |  6   |
| CC20190310 |   0_14  | 0.262947566677 |  7   |
| CC20190310 |   2_1

In [25]:
# Using scaled number of seconds

name = 'cosine' 
target = 'Scaled_SecNumF' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.284367s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 175000/178122 queries. users per second: 19207.8

recommendations finished on 176000/178122 queries. users per second: 19153

recommendations finished on 177000/178122 queries. users per second: 19081.6

recommendations finished on 178000/178122 queries. users per second: 18883.8

+------------+---------+-----------------+------+
|  Customer  | Product |      score      | rank |
+------------+---------+-----------------+------+
| CC20190310 |   1_21  | 0.0829681871254 |  1   |
| CC20190310 |   6_6   | 0.0779237492236 |  2   |
| CC20190310 |   5_8   | 0.0737923807491 |  3   |
| CC20190310 |   5_2   | 0.0734602505761 |  4   |
| CC20190310 |   2_3   | 0.0661373334813 |  5   |
| CC20190310 |   3_2   | 0.0649376490902 |  6   |
| CC20190310 |   4_0   | 0.0633963119088 |  7   |
| CC20190310 |   3_4   | 0.0633933054918 |  8   |
| CC20190310 |   5_14  | 0.0574642750569 |  9   |
| CC20190310 |   7_20  | 0.0565055016148 |  10  |
| CC20190310 |   1_21  | 0.0829681871254 |  1   |
| CC20190310 |   6_6   | 0.0779237492236 |  2   |
| CC20190310 |   5_8   | 0.0737923807491 |  3   |
| CC20190310 |   5_2   | 0.0734602505761 |  4   |
| CC20190310 |   2_3   | 0.0661373334813 |  5   |
| CC20190310 |   3_2   | 0.0649376490902 |  6   |
| CC20190310 |   4_0   | 0.0633963119088 |  7   |


#### 7.2. Pearson similarity

In [26]:
# Using number of seconds

name = 'pearson'
target = 'SecNumF'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.283299s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 175000/178122 queries. users per second: 17381.7

recommendations finished on 176000/178122 queries. users per second: 17285.2

recommendations finished on 177000/178122 queries. users per second: 17038.4

recommendations finished on 178000/178122 queries. users per second: 16903.5

+------------+---------+---------------+------+
|  Customer  | Product |     score     | rank |
+------------+---------+---------------+------+
| CC20190310 |   2_29  | 55.8069306931 |  1   |
| CC20190310 |   3_21  | 55.4766564911 |  2   |
| CC20190310 |   6_19  | 55.3986390801 |  3   |
| CC20190310 |   2_28  | 55.2346439167 |  4   |
| CC20190310 |   2_15  | 55.1720656599 |  5   |
| CC20190310 |   1_21  | 55.1268112012 |  6   |
| CC20190310 |   2_10  | 54.9693854988 |  7   |
| CC20190310 |   0_15  | 54.9043274274 |  8   |
| CC20190310 |   7_21  | 54.7591065611 |  9   |
| CC20190310 |   7_11  | 54.5520536838 |  10  |
| CC20190310 |   2_29  | 55.8069306931 |  1   |
| CC20190310 |   3_21  | 55.4766564911 |  2   |
| CC20190310 |   6_19  | 55.3986390801 |  3   |
| CC20190310 |   2_28  | 55.2346439167 |  4   |
| CC20190310 |   2_15  | 55.1720656599 |  5   |
| CC20190310 |   1_21  | 55.1268112012 |  6   |
| CC20190310 |   2_10  | 54.9693854988 |  7   |
| CC20190310 |   0_15  | 54.9043274274 |

In [27]:
# Using SecNumF dummy

name = 'pearson'
target = 'SecNumF_Dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns SecNumF;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.287466s

...

recommendations finished on 174000/178122 queries. users per second: 84341.3

recommendations finished on 175000/178122 queries. users per second: 84484.2

recommendations finished on 176000/178122 queries. users per second: 84588.9

recommendations finished on 177000/178122 queries. users per second: 84715

recommendations finished on 178000/178122 queries. users per second: 84575.1

In [28]:
# Using scaled number of seconds

name = 'pearson'
target = 'Scaled_SecNumF'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 142497 observations with 1436 users and 235 items.

Data prepared in: 0.267836s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 175000/178122 queries. users per second: 16670.5

recommendations finished on 176000/178122 queries. users per second: 16623.1

recommendations finished on 177000/178122 queries. users per second: 16597.4

recommendations finished on 178000/178122 queries. users per second: 16541.7

+------------+---------+----------------+------+
|  Customer  | Product |     score      | rank |
+------------+---------+----------------+------+
| CC20190310 |   6_27  | 0.24363958062  |  1   |
| CC20190310 |   1_27  | 0.218157582938 |  2   |
| CC20190310 |   4_0   | 0.211857996484 |  3   |
| CC20190310 |   0_28  | 0.205501183252 |  4   |
| CC20190310 |   2_27  | 0.203471035128 |  5   |
| CC20190310 |   6_8   | 0.200077251427 |  6   |
| CC20190310 |   5_27  | 0.194917409628 |  7   |
| CC20190310 |   1_26  | 0.190805694743 |  8   |
| CC20190310 |   7_20  | 0.189726399586 |  9   |
| CC20190310 |   3_4   | 0.187896649457 |  10  |
| CC20190310 |   6_27  | 0.24363958062  |  1   |
| CC20190310 |   1_27  | 0.218157582938 |  2   |
| CC20190310 |   4_0   | 0.211857996484 |  3   |
| CC20190310 |   0_28  | 0.205501183252 |  4   |
| CC20190310 |   2_27  | 0.203471035128 |  5   |
| CC20190310 |   6_8   | 0.200077251427 |  6   |
| CC20190310 |   5_27  | 0.194917409628 |  7   |
| CC20190310 |   1_2

## 8. Model Evaluation

In [29]:
# create initial callable variables for model evaluation:

models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on SecNumF', 'Cosine Similarity on SecNumF', 'Pearson Similarity on SecNumF']
names_w_dummy = ['Popularity Model on SecNumF Dummy', 'Cosine Similarity on SecNumF Dummy', 'Pearson Similarity on SecNumF Dummy']
names_w_norm = ['Popularity Model on Scaled SecNumF', 'Cosine Similarity on Scaled SecNumF', 'Pearson Similarity on Scaled SecNumF']

In [30]:
# Compare all the models we have built based on RMSE and precision-recall characteristics:

eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on SecNumF


recommendations finished on 1000/1435 queries. users per second: 74621.3


Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.366550522648 | 0.0112228685239 |
|   2    | 0.342508710801 | 0.0208615622934 |
|   3    | 0.329152148664 | 0.0306430345791 |
|   4    | 0.320209059233 | 0.0399430057306 |
|   5    | 0.314982578397 | 0.0492209490915 |
|   6    | 0.310801393728 | 0.0581818402709 |
|   7    | 0.309308113489 | 0.0679476217706 |
|   8    | 0.308885017422 | 0.0774706203979 |
|   9    | 0.308865660085 | 0.0870841022526 |
|   10   | 0.309128919861 | 0.0966855053331 |
+--------+----------------+-----------------+
[10 rows x 3 columns]


Overall RMSE: 48.620436495

Per User RMSE (best)
+-----------+-------+---------------+
|  Customer | count |      rmse     |
+-----------+-------+---------------+
| RC8602601 |   6   | 12.6912699816 |
+-----------+-------+---------------+
[1 rows x 3 columns]


Per User RMSE (wors

recommendations finished on 1000/1435 queries. users per second: 25583.3


Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.363763066202 | 0.0118415822491 |
|   2    | 0.354355400697 | 0.0225991744595 |
|   3    | 0.344715447154 |  0.03244358795  |
|   4    | 0.335714285714 |  0.041959266198 |
|   5    | 0.32543554007  | 0.0510344171372 |
|   6    | 0.315911730546 | 0.0594655535508 |
|   7    | 0.311199601792 | 0.0687521369334 |
|   8    | 0.306794425087 | 0.0778066866816 |
|   9    | 0.304529616725 | 0.0868440033379 |
|   10   | 0.305017421603 | 0.0966336724551 |
+--------+----------------+-----------------+
[10 rows x 3 columns]


Overall RMSE: 67.4889457856

Per User RMSE (best)
+-----------+-------+--------------+
|  Customer | count |     rmse     |
+-----------+-------+--------------+
| RC2621091 |   3   | 8.1021740761 |
+-----------+-------+--------------+
[1 rows x 3 columns]


Per User RMSE (worst)
+

...


Precision and recall summary statistics by cutoff
+--------+-----------------+------------------+
| cutoff |  mean_precision |   mean_recall    |
+--------+-----------------+------------------+
|   1    | 0.0557103064067 | 0.00206154484777 |
|   2    |  0.111768802228 | 0.0074973777594  |
|   3    |  0.130919220056 |  0.012818568576  |
|   4    |  0.142757660167 | 0.0194180898827  |
|   5    |  0.147632311978 | 0.0248644540302  |
|   6    |  0.156685236769 | 0.0314669066072  |
|   7    |  0.16723040191  | 0.0396122802673  |
|   8    |  0.174181754875 |  0.046914957932  |
|   9    |  0.18144537295  | 0.0549697054489  |
|   10   |  0.18795264624  | 0.0622969856805  |
+--------+-----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.156002314695

Per User RMSE (best)
+-----------+-------+-----------------+
|  Customer | count |       rmse      |
+-----------+-------+-----------------+
| RC8602601 |   11  | 0.0446702467514 |
+-----------+-------+-----------------+
[1

recommendations finished on 1000/1436 queries. users per second: 15545.3


Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.361420612813 | 0.0111937663647 |
|   2    | 0.354456824513 |  0.021651180006 |
|   3    | 0.35817084494  | 0.0331852234804 |
|   4    | 0.360550139276 | 0.0444501959992 |
|   5    | 0.358774373259 | 0.0547337487095 |
|   6    | 0.358751160631 | 0.0657281648182 |
|   7    | 0.358734580183 | 0.0767381996522 |
|   8    | 0.358983286908 |  0.087435755425 |
|   9    | 0.358789848344 | 0.0987462178937 |
|   10   | 0.359818941504 |  0.110137759649 |
+--------+----------------+-----------------+
[10 rows x 3 columns]


Overall RMSE: 0.197393983093

Per User RMSE (best)
+-----------+-------+------------------+
|  Customer | count |       rmse       |
+-----------+-------+------------------+
| RC1444594 |   3   | 0.00756841598641 |
+-----------+-------+------------------+
[1 rows x 3 columns]


Pe

recommendations finished on 1000/1436 queries. users per second: 15778.1


Precision and recall summary statistics by cutoff
+--------+----------------+------------------+
| cutoff | mean_precision |   mean_recall    |
+--------+----------------+------------------+
|   1    | 0.105849582173 | 0.00308714059606 |
|   2    | 0.151810584958 | 0.00836724132678 |
|   3    | 0.172237697307 | 0.0141321402712  |
|   4    | 0.189415041783 | 0.0219775561362  |
|   5    | 0.196935933148 | 0.0289382525183  |
|   6    | 0.202065923863 |  0.035606095255  |
|   7    | 0.204138479904 | 0.0418323929258  |
|   8    | 0.206998607242 | 0.0485299948224  |
|   9    | 0.21440730424  | 0.0568212676678  |
|   10   | 0.218454038997 | 0.0644291187727  |
+--------+----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.146440082046

Per User RMSE (best)
+-----------+-------+---------------+
|  Customer | count |      rmse     |
+-----------+-------+---------------+
| RC8602601 |   11  | 0.04251692902 |
+-----------+-------+---------------+
[1 rows x 3 columns]


Per

### From the above summary, we select the Cosine similarity on scaled number of seconds approach as our final model, because this combination gives the best results (the desirable outcome has low RMSE and precision-recall close to 1).

## 9. Final Output

In [32]:
# data_norm

In [31]:
# rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), 
                                            user_id=user_id, 
                                            item_id=item_id,
                                            target ='Scaled_SecNumF',
                                            similarity_type = 'cosine' )

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 178122 observations with 1436 users and 235 items.

Data prepared in: 0.314481s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 175000/178122 queries. users per second: 15336.9

recommendations finished on 176000/178122 queries. users per second: 15293.7

recommendations finished on 177000/178122 queries. users per second: 15208.9

recommendations finished on 178000/178122 queries. users per second: 15116.9

+------------+---------+------------------+------+
|  Customer  | Product |      score       | rank |
+------------+---------+------------------+------+
| CC20190310 |   1_21  | 0.0581188351609  |  1   |
| CC20190310 |   4_5   | 0.0405291745829  |  2   |
| CC20190310 |   1_7   | 0.0328840735347  |  3   |
| CC20190310 |   3_9   | 0.0271409228791  |  4   |
| CC20190310 |   5_14  | 0.0252305815386  |  5   |
| CC20190310 |   2_15  | 0.0249330373698  |  6   |
| CC20190310 |   7_21  | 0.0126251314962  |  7   |
| CC20190310 |   6_27  | 0.00319924188215 |  8   |
| CC20190310 |   0_29  |       0.0        |  9   |
| CC20190310 |   7_28  |       0.0        |  10  |
| CC20190310 |   1_21  | 0.0581188351609  |  1   |
| CC20190310 |   4_5   | 0.0405291745829  |  2   |
| CC20190310 |   1_7   | 0.0328840735347  |  3   |
| CC20190310 |   3_9   | 0.0271409228791  |  4   |
| CC20190310 |   5_14  | 0.0252305815386  |  5   |
| CC20190310 |   2_15  | 0.0249330373698  |  6   |
| CC20190310 |   7_21  | 0.0126

#### 9.1. CSV output file

In [32]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(1781220, 4)


,Customer,Product,score,rank
0,CC20190310,1_21,0.058119,1
1,CC20190310,4_5,0.040529,2
2,CC20190310,1_7,0.032884,3
3,CC20190310,3_9,0.027141,4
4,CC20190310,5_14,0.025231,5


In [33]:
# create the desired output

def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe().drop_duplicates()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['Customer', 'recommendedProducts']].drop_duplicates() \
        .sort_values('Customer').set_index('Customer')
    if print_csv:
        df_output.to_csv('../recommendation_list.csv')
        print("An output file was created with the name 'recommendation_list.csv'")
    return df_output

In [34]:
# setprint_csv to true to print out the output in a csv file

df_output = create_output(pear_norm, users_to_recommend, n_rec , print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/178122 queries. users per second: 23440.6

recommendations finished on 2000/178122 queries. users per second: 23758.9

recommendations finished on 3000/178122 queries. users per second: 24528.2

recommendations finished on 4000/178122 queries. users per second: 23900.9

recommendations finished on 5000/178122 queries. users per second: 23394.3

...

recommendations finished on 176000/178122 queries. users per second: 15887.1

recommendations finished on 177000/178122 queries. users per second: 15864.2

recommendations finished on 178000/178122 queries. users per second: 15793

An output file was created with the name 'recommendation_list.csv'
(1436, 1)


,recommendedProducts
Customer,
CC20190310,6_27|1_27|4_0|0_28|2_27|6_8|5_27|1_26|7_20|3_4
CC201903100,4_0|6_8|1_27|4_17|5_5|3_5|0_28|1_11|1_0|7_11
CC201903101,1_27|2_17|0_28|2_27|2_26|5_27|4_14|1_26|5_22|5_7
CC201903102,1_27|2_20|5_9|0_28|3_5|4_14|2_26|3_15|5_27|1_26
CC201903103,4_0|3_5|5_11|7_11|1_27|5_5|5_8|7_18|0_12|3_14


#### 9.2. Customer recommendation function

In [35]:
# return a recommendation list of ten products for a given customer ID

def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [36]:
# test example
customer_recomendation('CC20190310')

recommendedProducts    6_27|1_27|4_0|0_28|2_27|6_8|5_27|1_26|7_20|3_4
Name: CC20190310, dtype: object

## 10. Given a category Id, get a list of customers that are probably interested

In [12]:
# returns a list of target Customers that are probably interested in items of category "catId"

def findTargetCustomers(catId):
    
    # load the list of recommendations to customers
    customers = pd.read_csv('../recommendation_list.csv') 
    
    # Create a separate row for each one of the multiple values in column "recommendedProducts"
    def splitDataFrameList(df,target_column,separator):
        def splitListToRows(row,row_accumulator,target_column,separator):
            split_row = row[target_column].split(separator)
            for s in split_row:
                new_row = row.to_dict()
                new_row[target_column] = s
                row_accumulator.append(new_row)
        new_rows = []
        df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
        new_df = pd.DataFrame(new_rows)
        return new_df

    df = splitDataFrameList(customers,'recommendedProducts','|')
    
    # split the information from the recommendedProducts column in two new columns: Category and Item
    df[['Category', 'Item']] = df['recommendedProducts'].str.split('_',expand=True)
    df = df.drop('recommendedProducts',axis=1)
    
    # keep only the rows that refer to the given Category and to regular customers (begining with CC)
    df = df[(df['Category'] == catId) & (df['Customer'].str.startswith('CC'))]
    
    # Keep only the customer ids
    df = df['Customer']

    tarCustLst = df.drop_duplicates()
    tarCustLst.to_csv('../tarCustLst.csv',index = None, header=False)
    
    return tarCustLst

In [13]:
# test example
tarCustLst = findTargetCustomers('1')